<head>
  <meta name="author" content="Rogério de Oliveira">
  <meta institution="author" content="ITM">
</head>

<img src="https://maua.br/images/selo-60-anos-maua.svg" width=300, align="right">
<!-- <h1 align=left><font size = 6, style="color:rgb(200,0,0)"> optional title </font></h1> -->


# Lab: Validação Cruzada e GridSearch





# Caso: **Classificação de Tipos de Vidro para Reciclagem**

Nossa base de dados classifica vidros industrializados em 7 categorias conforme suas características químicas:

* Classe 1: janelas de construção (processadas com flutuação)
* Classe 2: janelas de construção (processadas sem flutuação)
* Classe 3: janelas do veículo (processadas com flutuação)
* Classe 4: janelas do veículo (processadas sem flutuação)
* Classe 5: recipientes
* Classe 6: talheres
* Classe 7: faróis

(*algumas dessas classes podem não estar presentes no data-set*).

Os dados estão na URL: https://github.com/Rogerio-mack/Machine-Learning-I/raw/main/data/glasses.csv

Aqui vai nos interessar classificar os vidros para efeito de reciclagem em 3 categorias:

* **C = Vidros de Construção**
* **V = Vidros de Veículos**
* **O = Outros**

E para isso vamos empregar uma seleção de hiperparâmetros de modelos com o GridSearch que você aprendeu na aula teórica.




In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# Aquisição dos dados

In [ ]:
df = pd.read_csv('https://github.com/Rogerio-mack/IMT_Ciencia_de_Dados/raw/main/data/glasses.csv')
df.head()
# df.shape

,Id number,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type of glass
0,1,1.52101,13.64,D,1.10,71.78,0.06,8.75,0.0,0.0,1
1,2,1.51761,13.89,D,1.36,72.73,0.48,7.83,0.0,0.0,1
2,3,1.51618,13.53,C,1.54,72.99,0.39,7.78,0.0,0.0,1
3,4,1.51766,13.21,D,1.29,72.61,0.57,8.22,0.0,0.0,1
4,5,1.51742,13.27,D,1.24,73.08,0.55,8.07,0.0,0.0,1


# Tratamento de Nulos

In [ ]:
# seu código
df.isnull().sum() / len(df)

Id number        0.0
RI               0.0
Na               0.0
Mg               0.0
Al               0.0
Si               0.0
K                0.0
Ca               0.0
Ba               0.0
Fe               0.0
Type of glass    0.0
dtype: float64

In [ ]:
df['Type of glass'].unique()

array([1, 2, 3, 5, 7])

# Tratamento do Atributo Target

In [ ]:
#@markdown check
all(( df['Type of glass'].value_counts() == [146,46,13] ) == True)

True

In [ ]:
# seu código
# for key, value in df['Type of glass'].items():
#   if value == 1 or value == 2:
#     df.at[key, 'Type of glass'] = "C"
#   elif value == 3 or value == 4 or value == 7:
#     df.at[key, 'Type of glass'] = "V"
#   elif value == 5 or value == 6:
#     df.at[key, 'Type of glass'] = "O"

df['Type of glass'] = df['Type of glass'].map({1: 'C', 2: 'C', 3: 'V', 4: 'V', 7: 'V', 5: 'O', 6: 'O'})

# df.head()
df['Type of glass'].unique()

array(['C', 'V', 'O'], dtype=object)

# Exclusão de atributos

In [ ]:
# seu código
df.drop('Type of glass', axis=1, inplace=True)
df.shape

(205, 10)

In [ ]:
#@markdown check
df.shape == (205,10)

True

# Hot Encode

In [ ]:
df.dtypes

Id number      int64
RI           float64
Na           float64
Mg            object
Al           float64
Si           float64
K            float64
Ca           float64
Ba           float64
Fe           float64
dtype: object

In [ ]:
# seu código
from sklearn.preprocessing import OneHotEncoder

hot_encode = OneHotEncoder(handle_unknown='ignore',sparse_output=False,drop='first')
hot_encode.fit(df.select_dtypes(exclude='number'))

df_hot_encode = pd.DataFrame(hot_encode.transform(df.select_dtypes(exclude='number')),columns=hot_encode.get_feature_names_out())
df_hot_encode.head()

df = pd.concat([df_hot_encode,df.select_dtypes('number')],axis=1)
df.head()

,Mg_B,Mg_C,Mg_D,Id number,RI,Na,Al,Si,K,Ca,Ba,Fe
0,0.0,0.0,1.0,1,1.52101,13.64,1.10,71.78,0.06,8.75,0.0,0.0
1,0.0,0.0,1.0,2,1.51761,13.89,1.36,72.73,0.48,7.83,0.0,0.0
2,0.0,1.0,0.0,3,1.51618,13.53,1.54,72.99,0.39,7.78,0.0,0.0
3,0.0,0.0,1.0,4,1.51766,13.21,1.29,72.61,0.57,8.22,0.0,0.0
4,0.0,0.0,1.0,5,1.51742,13.27,1.24,73.08,0.55,8.07,0.0,0.0


In [ ]:
#@markdown check
df[ [x for x in df.columns if x.find('Mg_') == 0] ].sum().sum() == 156

True

# Select Features?

In [ ]:
# opcional, não vamos excluir nenhum atributo aqui

# Normalização

Empregue `StandardScaler`.

In [ ]:
# seu código
from sklearn.preprocessing import StandardScaler, MinMaxScaler

scaler = StandardScaler()
scaler.fit(df)

df_scaled = scaler.transform(df)
df_scaled = pd.DataFrame(df_scaled)

df_scaled = pd.concat([df_scaled],axis=1)

df = df_scaled
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,-0.53033,-0.605530,1.631170,-1.689620,0.860330,0.377325,-0.702388,-1.127508,-0.699270,-0.133436,-0.361529,-0.603761
1,-0.53033,-0.605530,1.631170,-1.673259,-0.262715,0.706536,-0.178130,0.137804,-0.059245,-0.781542,-0.361529,-0.603761
2,-0.53033,1.651446,-0.613057,-1.656898,-0.735055,0.232472,0.184818,0.484099,-0.196393,-0.816765,-0.361529,-0.603761
3,-0.53033,-0.605530,1.631170,-1.640538,-0.246200,-0.188919,-0.319276,-0.022025,0.077903,-0.506801,-0.361529,-0.603761
4,-0.53033,-0.605530,1.631170,-1.624177,-0.325474,-0.109908,-0.420095,0.603971,0.047426,-0.612470,-0.361529,-0.603761


In [ ]:
#@markdown check
df[['Na','Si']].sum().sum() == 3.282707439211663e-12

KeyError: ignored

# Treinando o Modelo

Você vai treinar um modelo de Árvore de Decisão buscando os melhores hiperparâmetros de 'max_depth' e 'criterion' (pesquise os possíveis valores na documentação do scikit-learn). Entretanto, no lugar da acuracidade, você empregar o F1 score (`f1_macro`) que é uma métrica que balanceia os resultados de precisão e recall.

Aqui um checklist do que precisa ser feito...

1. Separe os dados de Treinamento e Teste empregandp 0.3 dos dados para teste, estratificados e não deixe de empregar o seed 123.

2. Defina uma DecisionTree como Estimador Base. Não deixe de empregar o parâmetro `random_state=123` no estimador base para a reprodutibilidade dos resultados.

3. Especifique o range dos valores 'max_depth'  de 3 a 10, e 'criterion' que você deseja empregar pesquisando os valores na documentação do scikit-learn.

4. Configure o `GridSearchCV` para empregar 5 partições e empregar o score de `f1_macro` para a seleção dos melhores hiperparâmetros.

5. Verifique os Resultados gerando um classification report para ver as métricas do modelo.


**Nota**: não use outros parâmetros ou recursos não especificados aqui.

In [ ]:
# seu código

In [ ]:
#@markdown check
if X_train.sum().sum() != -19.006240819955718:
  print('verifique train_test_split')
else:
  print(True)
if clf.get_params()['estimator__random_state'] != 123:
  print('verifique o random state da árvore de decisão')
else:
  print(True)
if clf.get_params()['scoring'] != 'f1_macro':
  print('verifique o score aplicado')
else:
  print(True)

True
True
True


# Predição de Novos Casos

Considere os casos abaixo.

In [ ]:
df_cases = pd.read_csv('https://github.com/Rogerio-mack/IMT_Ciencia_de_Dados/raw/main/data/glasses_test.csv')
df_cases.head()

,Id number,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe
0,0,1.515877,12.81,C,1.48,73.89,0.60,8.12,0.0,0.01
1,1,1.515627,12.89,C,1.52,74.10,0.67,7.83,0.0,0.01
2,2,1.518166,12.90,D,1.19,73.44,0.60,8.43,0.0,0.01
3,3,1.517456,13.33,B,1.52,73.04,0.58,8.79,0.0,0.01
4,4,1.514837,13.81,B,3.50,70.89,1.68,5.87,2.2,0.01


In [ ]:
# seu código